# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [45]:
# Instalar o selenium
# !pip install selenium

# baixar o webdriver e colocar ele na mesma pasta do código
# chromedriver baixar para navegador chrome

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
navegador = webdriver.Chrome('chromedriver.exe')


# Passo 1: Pegar cotação do dólar
# - Entrar no site do google
navegador.get('https://google.com.br')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação dólar")
# - Pesquisar a cotação do dolar
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# - pegar o valor do dolar
dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print('Dólar: {}' .format(dolar))


# Passo 2: Pegar a cotação do Euro
# - Entrar no site do google
navegador.get('https://google.com.br')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação EURO")
# - Pesquisar a cotação do dolar
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# - pegar o valor do dolar
euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print('Euro: {}' .format(euro))


# Passo 3: Pegar a cotação do Ouro
# - Entrar no site do melhor cambio
navegador.get('https://www.melhorcambio.com/ouro-hoje')
# - pegar o valor do dolar
ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
ouro = ouro.replace(',','.')
print('Ouro: {}' .format(ouro))




Dólar: 5.4939
Euro: 6.350838522
Ouro: 311.66


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [46]:
# Passo 4: Importar a base de dados

# Sempre que for importar uma base de dados para o python, importar o pandas
import pandas as pd
tabela = pd.read_excel('Produtos.xlsx')
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [47]:
# Passo 5: Atualizar a cotação, o preço de compra e o preço de venda

# atualizar cotação

# para filtrar a tabela utilizar tabela.loc
# tabela.loc[linha, coluna] = novo_valor
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] =  float(euro)
tabela.loc[tabela['Moeda'] == 'Ouro' ,'Cotação'] =  float(ouro)


# Atualizar preço de compra: preço original * cotação
tabela['Preço Base Reais'] = tabela['Preço Base Original'] * tabela['Cotação']

# atualizar o preço de venda = preço de compra * margem
tabela['Preço Final'] = tabela['Preço Base Reais'] * tabela['Margem']
tabela['Preço Final'] = tabela['Preço Final'].map('R${:.2f}'.format)
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.493900,5493.845061,1.40,R$7691.38
1,Carro Renault,4500.00,Euro,6.350839,28578.773349,2.00,R$57157.55
2,Notebook Dell,899.99,Dólar,5.493900,4944.455061,1.70,R$8405.57
3,IPhone,799.00,Dólar,5.493900,4389.626100,1.70,R$7462.36
4,Carro Fiat,3000.00,Euro,6.350839,19052.515566,1.90,R$36199.78
5,Celular Xiaomi,480.48,Dólar,5.493900,2639.709072,2.00,R$5279.42
6,Joia 20g,20.00,Ouro,311.660000,6233.200000,1.15,R$7168.18


### Agora vamos exportar a nova base de preços atualizada

In [48]:
# Passo 6: Exportar o relatório atualizado
tabela.to_excel("Produtos_Atualizado3.xlsx", index=False)
navegador.quit()